In [1]:
import hashlib # библиотека для реализации алгоритма хэширования 
import os # библиотека для работы с файловой системой
import time # библиотека для измерения времени
import psutil # библиотека для фиксации нагрузки на ОЗУ и ЦП 

Далее введём константы, определяющие количество битовых сдвигов. Они необходимы для выполнения основных операций алгоритма:

In [2]:
# Константы для MD5
S11 = 7
S12 = 12
S13 = 17
S14 = 22
S21 = 5
S22 = 9
S23 = 14
S24 = 20
S31 = 4
S32 = 11
S33 = 16
S34 = 23
S41 = 6
S42 = 10
S43 = 15
S44 = 21

Напишем функции для преобразования:

In [3]:
# Функция для преобразования строки в 32-битный блок
def string_to_block(block_bytes):
    block = [0] * 16
    for i in range(len(block_bytes)):
        block[i // 4] |= block_bytes[i] << ((3 - (i % 4)) * 8)
    return block

# Функция для преобразования 32-битного блока в строку
def block_to_string(block):
    string = ''
    for i in range(16):
        for j in range(4):
            string += chr((block[i] >> ((3 - j) * 8)) & 0xFF)
    return string

1. string_to_block: Изменяет строку в список из 16 32-битных целых чисел (блоков), разбивая строку на 4-байтовые блоки.
2. block_to_string: Изменяет список из 16 32-битных целых чисел (блоков) обратно в строку.

In [4]:
# Функция для левого циклического сдвига
def left_rotate(x, s):
    return ((x << s) | (x >> (32 - s))) & 0xFFFFFFFF

left_rotate: выполняет левый циклический сдвиг 32-битного целого числа x на s позиций

In [5]:
# Функция MD5
def md5(message):
    if isinstance(message, str):
        # Преобразуем сообщение в байты, если это строка
        message_bytes = bytes(message, 'utf-8')
    else:
        # Если это уже байты, просто используем message
        message_bytes = message

    # Преобразуем сообщение в блоки
    blocks = []
    for i in range(0, len(message_bytes), 64):
        block = string_to_block(message_bytes[i:i+64])
        blocks.append(block)

    # Дополняем последнее сообщение 
    last_block = blocks[-1]
    last_block[14] = int(len(message_bytes) * 8)
    blocks.append(last_block)

    # Инициализируем переменные состояния
    A = 0x67452301
    B = 0xEFCDAB89
    C = 0x98BADCFE
    D = 0x10325476

    # Основной цикл
    for block in blocks:
        # Копия блока
        M = [0] * 16
        for i in range(16):
            M[i] = block[i]

        # Цикл F
        for i in range(16):
            F = (B & C) | ((~B) & D)
            g = i
            S = S11
            temp = A + F + M[g] + 0xd76aa478
            A = left_rotate(temp, S) + B
            D = C
            C = B
            B = A

        # Цикл G
        for i in range(16, 32):
            F = (D & B) | ((~D) & C)
            g = (5 * i + 1) % 16
            S = S21
            temp = A + F + M[g] + 0xe8c7b756
            A = left_rotate(temp, S) + B
            D = C
            C = B
            B = A

        # Цикл H
        for i in range(32, 48):
            F = B ^ C ^ D
            g = (3 * i + 5) % 16
            S = S31
            temp = A + F + M[g] + 0x242070db
            A = left_rotate(temp, S) + B
            D = C
            C = B
            B = A

        # Цикл I
        for i in range(48, 64):
            F = C ^ (D | (~B))
            g = (7 * i) % 16
            S = S41
            temp = A + F + M[g] + 0xc19bf174
            A = left_rotate(temp, S) + B
            D = C
            C = B
            B = A

        # Обновляем переменные состояния
        A = (A + 0x67452301) & 0xFFFFFFFF
        B = (B + 0xEFCDAB89) & 0xFFFFFFFF
        C = (C + 0x98BADCFE) & 0xFFFFFFFF
        D = (D + 0x10325476) & 0xFFFFFFFF

    # Форматируем хэш
    hash_str = "%08x%08x%08x%08x" % (A, B, C, D)
    return hash_str

Функция md5 - это главная часть алгоритма. Что она делает:

1. Разбивает входное сообщение на блоки по 64 байта (blocks).
2. Дополняет последний блок, чтобы его длина была кратна 512 битам (last_block).
3. Инициализирует переменные состояния A, B, C, D (четыре 32-битных слова) значениями, определенными в стандарте MD5.
4. Выполняет четыре раунда обработки (F, G, H, I) над каждым блоком, используя циклический сдвиг, булевы операции и сложение по модулю.
5. Обновляет значения переменных состояния после каждого блока.
6. Форматирует окончательный хэш в 128-битное шестнадцатеричное число.

In [6]:
# Алгоритм диверсификации ключа
def diversify_key(key, salt):
    # Преобразование ключа и соли в байты
    key_bytes= bytes(key, 'utf-8')
    salt_bytes = bytes(salt, 'utf-8')

    # Конкатенация ключа и соли
    combined_bytes = key + salt
 
    # Вычисление MD5-хэша
    hash_value = md5(combined_bytes)

    # Преобразование хэша в шестнадцатеричный формат
    return hash_value

Преобразование ключа и соли в байты:

1. Преобразуем ключ key в байты, используя bytes(key, 'utf-8').
2. Преобразуем соль salt в байты, используя bytes(salt, 'utf-8').

Конкатенация ключа и соли: объединяем байты ключа и соли с помощью bytes(key, 'utf-8') + bytes(salt, 'utf-8').

Вычисление MD5-хэша: используем алгоритм MD5 для вычисления хэша полученной конкатенации.

Преобразование хэша в строку: преобразуем полученный хэш в шестнадцатеричный формат, чтобы его можно было использовать в качестве диверсифицированного ключа.

Cоль (salt) - это дополнительная строка, добавляемая к ключу (key) перед хэшированием в функции diversify_key.

#### Зачем нужна соль?

Соль используется для повышения безопасности алгоритма хэширования. Она делает хэширование более устойчивым к атакам по словарю.

#### Как работает соль?

1. Атаки по словарю: В этих атаках злоумышленник создает таблицу с хэшами распространенных паролей.

2. Без соли: Если пароль хэшируется без соли, злоумышленник может легко проверить, совпадает ли хэш атакуемого пароля с одним из хэшей в таблице.

3. С солью: Соль добавляет к ключу случайный элемент, делая хэш уникальным даже для одинаковых паролей.
Вместо того, чтобы хэшировать только пароль, алгоритм хэширует пароль + соль.
Злоумышленник не сможет использовать таблицу с хэшами паролей, потому что хэширование включает случайную соль.

Далее пишем функции для хэширования:

In [7]:
# Функция для хэширования блока данных
def hash_block(block, key):
    # Преобразуем ключ в байты
    key = key.encode()
    return md5(key + block.encode())

# Функция для хэширования файла
def hash_file(filename, key):
    with open(filename, 'rb') as f:
        file_content = f.read()

    # Чтение ключевого материала как байтов
    with open("key_material.txt", 'rb') as f: 
        key_material = f.read().strip()  # Убираем пробелы и преобразуем в байты

    return md5(key_material + file_content)  


# Функция для хэширования директории (рекурсивно)
def hash_directory(directory, key):
    hash_values = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            hash_values.append(hash_file(file_path, key))
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            hash_values.extend(hash_directory(dir_path, key))
    return hash_values

1. hash_block: объединяет ключ key с блоком данных block и вычисляет MD5-хэш результата.

2. hash_file: считывает содержимое файла filename, объединяет ключ key с содержимым файла и вычисляет MD5-хэш результата.

3. hash_directory: рекурсивно переходит по каталогу directory, хэширует каждый файл с помощью hash_file и каждую поддиректорию с помощью hash_directory, возвращает список хэшей для всех файлов и поддиректорий.

In [8]:
# Генерация ключевого материала
def generate_key_material(filename, key_length=32):
    key_material = os.urandom(key_length)
    with open(filename, 'wb') as f:
        f.write(key_material.hex().encode())  # Записываем в шестнадцатеричном формате

generate_key_material: генерирует случайный байтовый массив (key_material) заданной длины (key_length) и сохраняет этот массив в файл filename.

In [9]:
# Функция для измерения времени выполнения
def time_hashing(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

# Хэширование блоков данных
key = "SecretKey"

print("Хэширование блоков данных:")
for block_size in [1, 10**3, 10**6]:
    # Измеряем нагрузку перед хэшированием
    cpu_usage_before = psutil.cpu_percent(interval=1)
    memory_usage_before = psutil.virtual_memory().percent
    
    data_block = "a" * block_size
    block_hash, duration = time_hashing(hash_block, data_block, key)
    
    # Измеряем нагрузку после хэширования
    cpu_usage_after = psutil.cpu_percent(interval=1)
    memory_usage_after = psutil.virtual_memory().percent

    print(f"  Блок размером {block_size} байт: {duration:.6f} сек")
    print(f"    Изменение ЦП: {cpu_usage_after - cpu_usage_before}%")
    print(f"    Изменение ОЗУ: {memory_usage_after - memory_usage_before}%")

# Хэширование файлов
print("\nХэширование файлов:")
for file_size in [1, 100, 1000]:
    # Измеряем нагрузку перед хэшированием
    cpu_usage_before = psutil.cpu_percent(interval=1)
    memory_usage_before = psutil.virtual_memory().percent

    file_content = "a" * (file_size * 1024 * 1024)
    with open(f"temp_file_{file_size}mb.txt", 'wb') as f:
        f.write(file_content.encode())
    file_hash, duration = time_hashing(hash_file, f"temp_file_{file_size}mb.txt", key)
    
    # Измеряем нагрузку после хэширования
    cpu_usage_after = psutil.cpu_percent(interval=1)
    memory_usage_after = psutil.virtual_memory().percent

    print(f"  Файл размером {file_size} Мб: {duration:.2f} сек")
    print(f"    Изменение ЦП: {cpu_usage_after - cpu_usage_before}%")
    print(f"    Изменение ОЗУ: {memory_usage_after - memory_usage_before}%")
    os.remove(f"temp_file_{file_size}mb.txt")

Хэширование блоков данных:
  Блок размером 1 байт: 0.000999 сек
    Изменение ЦП: -7.6%
    Изменение ОЗУ: -0.10000000000000142%
  Блок размером 1000 байт: 0.006013 сек
    Изменение ЦП: -10.7%
    Изменение ОЗУ: -0.19999999999999574%
  Блок размером 1000000 байт: 1.639045 сек
    Изменение ЦП: -2.8000000000000003%
    Изменение ОЗУ: 0.10000000000000142%

Хэширование файлов:
  Файл размером 1 Мб: 1.69 сек
    Изменение ЦП: 9.3%
    Изменение ОЗУ: 0.20000000000000284%
  Файл размером 100 Мб: 190.98 сек
    Изменение ЦП: -10.5%
    Изменение ОЗУ: 3.1000000000000014%
  Файл размером 1000 Мб: 7892.58 сек
    Изменение ЦП: -2.299999999999999%
    Изменение ОЗУ: -27.799999999999997%


In [ ]:
# Функция для измерения времени выполнения
def time_hashing(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

# Хэширование блоков данных
key = "SecretKey"

print("Хэширование блоков данных:")
for block_size in [1, 10**3, 10**6]:
    # Измеряем нагрузку перед хэшированием
    cpu_usage_before = psutil.cpu_percent(interval=5)  # Увеличиваем интервал
    memory_usage_before = psutil.virtual_memory().percent

    data_block = "a" * block_size
    block_hash, duration = time_hashing(hash_block, data_block, key)

    # Измеряем нагрузку после хэширования
    cpu_usage_after = psutil.cpu_percent(interval=5)  # Увеличиваем интервал
    memory_usage_after = psutil.virtual_memory().percent

    print(f"  Блок размером {block_size} байт: {duration:.6f} сек")
    print(f"    Изменение ЦП: {cpu_usage_after - cpu_usage_before}%")
    print(f"    Изменение ОЗУ: {memory_usage_after - memory_usage_before}%")

# Хэширование файлов
print("\nХэширование файлов:")
for file_size in [1, 100, 1000]:
    # Измеряем нагрузку перед хэшированием
    cpu_usage_before = psutil.cpu_percent(interval=5)  # Увеличиваем интервал
    memory_usage_before = psutil.virtual_memory().percent

    file_content = "a" * (file_size * 1024 * 1024)
    with open(f"temp_file_{file_size}mb.txt", 'wb') as f:
        f.write(file_content.encode())
    file_hash, duration = time_hashing(hash_file, f"temp_file_{file_size}mb.txt", key)

    # Измеряем нагрузку после хэширования
    cpu_usage_after = psutil.cpu_percent(interval=5)  # Увеличиваем интервал
    memory_usage_after = psutil.virtual_memory().percent

    print(f"  Файл размером {file_size} Мб: {duration:.2f} сек")
    print(f"    Изменение ЦП: {cpu_usage_after - cpu_usage_before}%")
    print(f"    Изменение ОЗУ: {memory_usage_after - memory_usage_before}%")
    os.remove(f"temp_file_{file_size}mb.txt")

Хэширование блоков данных:
  Блок размером 1 байт: 0.000000 сек
    Изменение ЦП: -8.500000000000002%
    Изменение ОЗУ: 0.30000000000000426%
  Блок размером 1000 байт: 0.012991 сек
    Изменение ЦП: 0.10000000000000053%
    Изменение ОЗУ: 0.0%
  Блок размером 1000000 байт: 1.823951 сек
    Изменение ЦП: -5.6%
    Изменение ОЗУ: -0.20000000000000284%

Хэширование файлов:
  Файл размером 1 Мб: 1.64 сек
    Изменение ЦП: -0.7%
    Изменение ОЗУ: -0.6000000000000014%
  Файл размером 100 Мб: 187.06 сек
    Изменение ЦП: -4.7%
    Изменение ОЗУ: -1.2999999999999972%


1. Функция time_hashing(func, *args): Эта функция принимает функцию (func) и ее аргументы (*args). Она записывает начальное время, вызывает функцию с переданными аргументами, записывает конечное время и возвращает результат функции и продолжительность выполнения.


2. Измерение времени хэширования блоков: Используется цикл for, чтобы перебирать три размера блока (1, 10^3, 10^6 байт). Создается блок данных с помощью data_block = "a" * block_size. Вызывается функция time_hashing для hash_block с блоком данных и ключом. Выводится время выполнения с точностью до шести знаков после запятой. 


3. Измерение времени хэширования файлов: Используется цикл for, чтобы перебирать три размера файла (1, 100, 1000 Мб). Создается временный файл с помощью with open(...) и записывается в него содержимое (file_content = "a" * (file_size * 1024 * 1024)). Вызывается функция time_hashing для hash_file с именем файла и ключом. Выводится время выполнения с точностью до двух знаков после запятой. Удаляется временный файл с помощью os.remove(...).

In [36]:
# Примеры использования
if __name__ == '__main__':
    # Хэширование блока данных
    data_block = "Привет, мир!"
    key = "SecretKey"
    block_hash = hash_block(data_block, key)
    print(f"Хэш блока: {block_hash}")
    
    # Генерация ключевого материала (перед чтением из файла)
    generate_key_material("key_material.txt")

    # Хэширование файла
    file_path = "Рональд.txt"
    file_hash = hash_file(file_path, key)
    print(f"Хэш файла: {file_hash}")

    # Хэширование директории
    directory_path = "Проект"
    directory_hashes = hash_directory(directory_path, key)
    print(f"Хэши файлов в директории: {directory_hashes}")

    # Диверсификация ключа
    salt = "SaltValue"
    diversified_key = diversify_key(key, salt)
    print(f"Диверсифицированный ключ: {diversified_key}")

    # Генерация ключевого материала
    print("Ключевой материал сгенерирован в key_material.txt")

Хэш блока: ba4240a242cac92a66b29a45b9def5f4
Хэш файла: 0b0a954f93931dd7c96bd45040e34c03
Хэши файлов в директории: ['b6d5ff493f5e87d1e84bb9720953d6fa', '0b0a954f93931dd7c96bd45040e34c03', '5e83e86fe70c70f74d06f7f298caa050']
Диверсифицированный ключ: 404e32f1c8d6bb79eeb4aafd662c2280
Ключевой материал сгенерирован в key_material.txt


В блоке с if показано, как использовать функции хэширования, диверсификации ключа и генерации ключевого материала.